Special considerations

* The big question is: Do you __care__ about timing or do you just care about zeta? If you care about timing then Ne, mutation rate, sequence length and number of loci per population all matter.
* When creating empirical SFS files you _should_ use the `-a` flag of easySFS to include ALL snps per locus (unless you really know you don't need it).
* The PTA.DemographicModel `num_replicates` parameter controls the number of loci to simulate. This can be a little tricky because the number of loci for each simulated population should match the number of loci in each empirical population.
* There is a hidden parameter PTA.DemographicModel.\_hackersonly["proportional_msfs"] which controls whether the bins of the SFS are absolute counts or proportions (default is counts). Setting proportional_msfs to True is kind of a hack to work around simulating _exact_ numbers of loci with the exactly right mutation rate and so on. With `proportional_msfs` set to True we are really just comparing the _shapes_ of the msfs, which is _kind_ of what we are most interested in. This also allows to set `num_replicates` to a medium sized number (e.g. 1000), and also to use the same value for all populations.
* How important is having variable Ne per expanding population? What about variable epsilon (different expansion magnitudes)? Right now all taxa receive independent Ne, but co-expanding taxa share an epsilon value (expanding taxa that are not part of the co-expansion pulse have independent epsilon). Fixing Ne and epsilon to a single value across all taxa will improve the cross-validation simulation results, but may not fit the real data too well.
* Do we want to __estimate__ the magnitude of the size change? Or fix/parameterize this based on the stairwayplot analyses?
* Do the read lengths differ across these datasets? I assume so.

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import glob
import ipyparallel as ipp
import momi
import os
import pandas as pd
import PTA

from IPython.display import display

pd.set_option('display.max_columns', None)

ipyclient = ipp.Client(cluster_id="ipyrad")
print(len(ipyclient))

# For this to work dadi must be installed
easySFS = "/home/isaac/easySFS/easySFS.py"
!which $easySFS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
40
/home/isaac/easySFS/easySFS.py


In [16]:
# Set up some directories
prefix = "/home/isaac/media4TB/PTA-EasternSnakes/"
analysis_dir = prefix + "analysis/"
pops_dir = prefix + "East_snakes_PTA_input/pops_files/"
vcf_dir = prefix + "East_snakes_PTA_input/vcf_files/"
sfs_dir = prefix + "East_snakes_PTA_input/sfs_files/"

# Populations files for each pop were made with the "make_pops_SFS_stairway script"
all_pops = glob.glob(pops_dir + "*")
all_pops = sorted([x.split("/")[-1].rsplit(".", 1)[0] for x in all_pops])

all_pops

['Acontortrix_p123_v2_25miss_popAcontortrix',
 'Dpunctatus_p123_v3_25missEAST_popDpunctatus',
 'Lgetula_p123_v2_25miss_popk2east',
 'Lgetula_p123_v2_25miss_popk4get',
 'Lgetula_p123_v2_25miss_popk4holbnig',
 'Mflagellum_p123_v3_25missEast_popeast',
 'Pguttatus_p123_v2_25miss_popemor',
 'Pguttatus_p123_v2_25miss_popgut',
 'Sdekayi_p123_v2_25miss_popSdekayi',
 'abacura_only_popeast',
 'abacura_only_popwest',
 'erytro_poperytro',
 'milks_denovo-92_popelap',
 'milks_denovo-92_poptri']

In [11]:
expanding_pops = ["abacura_only_popeast",
                    "Acontortrix_p123_v2_25miss_popAcontortrix",
                    "Dpunctatus_p123_v3_25missEAST_popDpunctatus",
                    "Lgetula_p123_v2_25miss_popk2east",
                    "Lgetula_p123_v2_25miss_popk4get",
                    "Lgetula_p123_v2_25miss_popk4holbnig",
                    "Mflagellum_p123_v3_25missEast_popeast",
                    "milks_denovo-92_popelap",
                    "milks_denovo-92_poptri",
                    "Pguttatus_p123_v2_25miss_popgut"]
bottleneck_pops = ["abacura_only_popwest",
                    "erytro_poperytro",
                    "Pguttatus_p123_v2_25miss_popemor",
                    "Sdekayi_p123_v2_25miss_popSdekayi"]
print(expanding_pops)
print(bottleneck_pops)

['abacura_only_popeast', 'Acontortrix_p123_v2_25miss_popAcontortrix', 'Dpunctatus_p123_v3_25missEAST_popDpunctatus', 'Lgetula_p123_v2_25miss_popk2east', 'Lgetula_p123_v2_25miss_popk4get', 'Lgetula_p123_v2_25miss_popk4holbnig', 'Mflagellum_p123_v3_25missEast_popeast', 'milks_denovo-92_popelap', 'milks_denovo-92_poptri', 'Pguttatus_p123_v2_25miss_popgut']
['abacura_only_popwest', 'erytro_poperytro', 'Pguttatus_p123_v2_25miss_popemor', 'Sdekayi_p123_v2_25miss_popSdekayi']


In [10]:
## prototype the easySFS preview call for one population
pop = "Mflagellum_p123_v3_25missEast_popeast" # define the population
in_vcf = vcf_dir + pop + ".vcf" # define the vcf file location
pop_file = pops_dir + pop + ".txt" # define the population file location
print(in_vcf)
print(pop_file)

!$easySFS -i "$in_vcf" -p "$pop_file" -a --preview # run easySFS preview

/home/isaac/media4TB/PTA-EasternSnakes/East_snakes_PTA_input/vcf_files/Mflagellum_p123_v3_25missEast_popeast.vcf
/home/isaac/media4TB/PTA-EasternSnakes/East_snakes_PTA_input/pops_files/Mflagellum_p123_v3_25missEast_popeast.txt
Processing 1 populations - odict_keys(['east'])

    Running preview mode. We will print out the results for # of segregating sites
    for multiple values of projecting down for each population. The dadi
    manual recommends maximizing the # of seg sites for projections, but also
    a balance must be struck between # of seg sites and sample size.

    For each population you should choose the value of the projection that looks
    best and then rerun easySFS with the `--proj` flag.
    
east
(2, 19660)	(3, 29401)	(4, 36806)	(5, 42673)	(6, 48225)	(7, 53082)	(8, 57796)	(9, 61895)	(10, 66107)	(11, 68877)	(12, 72728)	(13, 71088)	(14, 74566)	(15, 46449)	(16, 48421)	(17, 21200)	(18, 21997)	




In [12]:
# Get easySFS preview results for all populations
preview_dict = {}
for pop in all_pops:
    print(pop)
    in_vcf = vcf_dir + pop + ".vcf"
    pop_file = pops_dir + pop + ".txt"
    preview_dict[pop] = !$easySFS -i "$in_vcf" -p "$pop_file" -a --preview

Acontortrix_p123_v2_25miss_popAcontortrix
Dpunctatus_p123_v3_25missEAST_popDpunctatus
Lgetula_p123_v2_25miss_popk2east
Lgetula_p123_v2_25miss_popk4get
Lgetula_p123_v2_25miss_popk4holbnig
Mflagellum_p123_v3_25missEast_popeast
Pguttatus_p123_v2_25miss_popemor
Pguttatus_p123_v2_25miss_popgut
Sdekayi_p123_v2_25miss_popSdekayi
abacura_only_popeast
abacura_only_popwest
erytro_poperytro
milks_denovo-92_popelap
milks_denovo-92_poptri


In [13]:
for k, v in preview_dict.items():
    print(k, "\n", v[-3], "\n")

Acontortrix_p123_v2_25miss_popAcontortrix 
 (2, 6528)	(3, 9771)	(4, 12355)	(5, 14585)	(6, 16629)	(7, 18502)	(8, 20313)	(9, 21991)	(10, 23667)	(11, 25266)	(12, 26853)	(13, 28392)	(14, 29918)	(15, 31397)	(16, 32875)	(17, 34307)	(18, 35747)	(19, 37142)	(20, 38551)	(21, 39933)	(22, 41317)	(23, 42668)	(24, 44030)	(25, 45355)	(26, 46697)	(27, 47997)	(28, 49322)	(29, 50609)	(30, 51917)	(31, 53171)	(32, 54465)	(33, 55686)	(34, 56967)	(35, 58163)	(36, 59431)	(37, 60609)	(38, 61865)	(39, 62999)	(40, 64244)	(41, 65237)	(42, 66470)	(43, 67119)	(44, 68337)	(45, 68133)	(46, 69331)	(47, 66878)	(48, 68026)	(49, 51807)	(50, 52660)	(51, 38546)	(52, 39158)	(53, 27102)	(54, 27514)	(55, 17960)	(56, 18226)	(57, 10641)	(58, 10793)	(59, 5530)	(60, 5607)	(61, 2182)	(62, 2210)	(63, 563)	(64, 570)	 

Dpunctatus_p123_v3_25missEAST_popDpunctatus 
 (2, 11142)	(3, 16589)	(4, 20969)	(5, 24705)	(6, 28135)	(7, 31214)	(8, 34151)	(9, 36732)	(10, 39343)	(11, 41705)	(12, 44080)	(13, 46266)	(14, 48459)	(15, 50506)	(16, 5255

In [ ]:
proj_dict = {}
## Expanding
proj_dict["abacura_only_popeast"] = 44
proj_dict["Acontortrix_p123_v2_25miss_popAcontortrix"] = 46
proj_dict["Dpunctatus_p123_v3_25missEAST_popDpunctatus"] = 52
proj_dict["Mflagellum_p123_v3_25missEast_popeast"] = 14
proj_dict["milks_denovo-92_popelap"] = 42
proj_dict["milks_denovo-92_poptri"] = 50
proj_dict["Pguttatus_p123_v2_25miss_popgut"] = 24

proj_dict["Lgetula_p123_v2_25miss_popk2east"] = 50
proj_dict["Lgetula_p123_v2_25miss_popk4get"] = 8
proj_dict["Lgetula_p123_v2_25miss_popk4holbnig"] = 44

## Contracting
proj_dict["abacura_only_popwest"] = 20
proj_dict["erytro_poperytro"] = 23
proj_dict["Pguttatus_p123_v2_25miss_popemor"] = 30
proj_dict["Sdekayi_p123_v2_25miss_popSdekayi"] = 40



In [15]:
proj = 8

# Get easySFS proj results for all populations
# <10 minutes
for pop in all_pops:
    print(pop)
    in_vcf = vcf_dir + pop + ".vcf"
    pop_file = pops_dir + pop + ".txt"
    pop_sfs_dir = sfs_dir + pop
    if not os.path.exists(pop_sfs_dir):
        os.makedirs(pop_sfs_dir)
    !$easySFS -i $in_vcf -p $pop_file -a --proj $proj -o $pop_sfs_dir -f

Acontortrix_p123_v2_25miss_popAcontortrix
Processing 1 populations - odict_keys(['Acontortrix'])
Doing 1D sfs - Acontortrix
Doing multiSFS for all pops
Dpunctatus_p123_v3_25missEAST_popDpunctatus
Processing 1 populations - odict_keys(['Dpunctatus'])
Doing 1D sfs - Dpunctatus
Doing multiSFS for all pops
Lgetula_p123_v2_25miss_popk2east
Processing 1 populations - odict_keys(['k2east'])
Doing 1D sfs - k2east
Doing multiSFS for all pops
Lgetula_p123_v2_25miss_popk4get
Processing 1 populations - odict_keys(['k4get'])
Doing 1D sfs - k4get
Doing multiSFS for all pops
Lgetula_p123_v2_25miss_popk4holbnig
Processing 1 populations - odict_keys(['k4holbnig'])
Doing 1D sfs - k4holbnig
Doing multiSFS for all pops
Mflagellum_p123_v3_25missEast_popeast
Processing 1 populations - odict_keys(['east'])
Doing 1D sfs - east
Doing multiSFS for all pops
Pguttatus_p123_v2_25miss_popemor
Processing 1 populations - odict_keys(['emor'])
Doing 1D sfs - emor
Doing multiSFS for all pops
Pguttatus_p123_v2_25miss_pop

In [28]:
# Drop the eastern getula population because we will include
# the 2 split k=4 populations (get & holbnig)
expanding_pops.remove("Lgetula_p123_v2_25miss_popk2east")

sfs_dict = {}
for pop in all_pops:
    pop_sfs_dir = sfs_dir + pop
    # Load from the dadi formatted sfs file generated by easySFS
    sfs_file = glob.glob(pop_sfs_dir + "/dadi/*.sfs")[-1]
    #sfs_file = pop_sfs_dir + "/dadi/{}.sfs".format(pop)
    sfs_dict[pop] = momi.data.convert.sfs_from_dadi(sfs_file)
exp_sfs = [sfs_dict[x] for x in expanding_pops]
bot_sfs = [sfs_dict[x] for x in bottleneck_pops]
len(exp_sfs)

9

In [31]:
sorted_expanding_msfs = PTA.msfs.multiSFS(exp_sfs, sort=True, proportions=True)
display(expanding_msfs.df)
display(expanding_msfs.to_dataframe())
expanding_msfs.dump(out=analysis_dir + "msfs/proj8-sort-props.msfs")

unsorted_expanding_msfs = PTA.msfs.multiSFS(exp_sfs, sort=False, proportions=True)
display(expanding_msfs.df)
display(expanding_msfs.to_dataframe())
expanding_msfs.dump(out=analysis_dir + "msfs/proj8-unsort-props.msfs")

,pop0,pop1,pop2,pop3,pop4,pop5,pop6,pop7,pop8
[[7_1]],0.713179,0.688062,0.678713,0.652572,0.650897,0.562070,0.554598,0.513267,0.508127
[[6_2]],0.239843,0.230946,0.226319,0.223852,0.194954,0.175918,0.172104,0.168170,0.134599
[[5_3]],0.174959,0.173487,0.152894,0.147902,0.119381,0.107057,0.106283,0.103255,0.095835
[[4_4]],0.080827,0.078543,0.068655,0.063709,0.055943,0.048967,0.047091,0.046833,0.040184


,pop0-[[7_1]],pop0-[[6_2]],pop0-[[5_3]],pop0-[[4_4]],pop1-[[7_1]],pop1-[[6_2]],pop1-[[5_3]],pop1-[[4_4]],pop2-[[7_1]],pop2-[[6_2]],pop2-[[5_3]],pop2-[[4_4]],pop3-[[7_1]],pop3-[[6_2]],pop3-[[5_3]],pop3-[[4_4]],pop4-[[7_1]],pop4-[[6_2]],pop4-[[5_3]],pop4-[[4_4]],pop5-[[7_1]],pop5-[[6_2]],pop5-[[5_3]],pop5-[[4_4]],pop6-[[7_1]],pop6-[[6_2]],pop6-[[5_3]],pop6-[[4_4]],pop7-[[7_1]],pop7-[[6_2]],pop7-[[5_3]],pop7-[[4_4]],pop8-[[7_1]],pop8-[[6_2]],pop8-[[5_3]],pop8-[[4_4]]
0,0.713179,0.239843,0.174959,0.080827,0.688062,0.230946,0.173487,0.078543,0.678713,0.226319,0.152894,0.068655,0.652572,0.223852,0.147902,0.063709,0.650897,0.194954,0.119381,0.055943,0.56207,0.175918,0.107057,0.048967,0.554598,0.172104,0.106283,0.047091,0.513267,0.16817,0.103255,0.046833,0.508127,0.134599,0.095835,0.040184


,pop0,pop1,pop2,pop3,pop4,pop5,pop6,pop7,pop8
[[7_1]],0.713179,0.688062,0.678713,0.652572,0.650897,0.562070,0.554598,0.513267,0.508127
[[6_2]],0.239843,0.230946,0.226319,0.223852,0.194954,0.175918,0.172104,0.168170,0.134599
[[5_3]],0.174959,0.173487,0.152894,0.147902,0.119381,0.107057,0.106283,0.103255,0.095835
[[4_4]],0.080827,0.078543,0.068655,0.063709,0.055943,0.048967,0.047091,0.046833,0.040184


,pop0-[[7_1]],pop0-[[6_2]],pop0-[[5_3]],pop0-[[4_4]],pop1-[[7_1]],pop1-[[6_2]],pop1-[[5_3]],pop1-[[4_4]],pop2-[[7_1]],pop2-[[6_2]],pop2-[[5_3]],pop2-[[4_4]],pop3-[[7_1]],pop3-[[6_2]],pop3-[[5_3]],pop3-[[4_4]],pop4-[[7_1]],pop4-[[6_2]],pop4-[[5_3]],pop4-[[4_4]],pop5-[[7_1]],pop5-[[6_2]],pop5-[[5_3]],pop5-[[4_4]],pop6-[[7_1]],pop6-[[6_2]],pop6-[[5_3]],pop6-[[4_4]],pop7-[[7_1]],pop7-[[6_2]],pop7-[[5_3]],pop7-[[4_4]],pop8-[[7_1]],pop8-[[6_2]],pop8-[[5_3]],pop8-[[4_4]]
0,0.713179,0.239843,0.174959,0.080827,0.688062,0.230946,0.173487,0.078543,0.678713,0.226319,0.152894,0.068655,0.652572,0.223852,0.147902,0.063709,0.650897,0.194954,0.119381,0.055943,0.56207,0.175918,0.107057,0.048967,0.554598,0.172104,0.106283,0.047091,0.513267,0.16817,0.103255,0.046833,0.508127,0.134599,0.095835,0.040184
